In [1]:
import os
import spacy
import spacy.attrs
nlp = spacy.load('en_core_web_md')

# prepare data
directory = 'output/unprocessed/body/'
documents = []
docdict = {}

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file

    if (os.path.isfile(f) and (".DS_Store" not in f)):
        # print(f)
        with open(f, 'r', encoding="utf8") as fp:
            currentfiledata = fp.read()
        documents.append(currentfiledata.lower())
        fnameparts = f.split("/")
        fid = fnameparts[len(fnameparts)-1].replace(".txt","")
        docdict[fid] = currentfiledata.lower()

key_list = list(docdict.keys())
val_list = list(docdict.values())

In [2]:
models = []        
for doc in documents:
    tmpdoc = ""
    m = None
    if (len(doc) > 1000000):
        tmpdoc = doc[:999999]
        m = nlp(tmpdoc)
    else:
        m = nlp(doc)
    position = val_list.index(doc)
    entry = {}
    entry["id"] = position
    print(position)
    entry["model"] = m
    models.append(entry)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [3]:
def freq(stri, word):
    # break the string into list of words
    str_list = stri.split()
    # gives set of unique words
    unique_words = set(str_list)
    return str_list.count(word)+1

In [4]:
rows = []
for model in models:
    print("Document ", model["id"], ": ")
    print("--------------")
    try:
        for entity in list(set(model["model"].ents)):
                ent_freq = freq(docdict[str(model["id"])], entity.text)
                if (((entity.label_ == "ORG") or (entity.label_ == "GPE") or (entity.label_ == "PERSON")) and (ent_freq > 0)):
                    row = []
                    row.append(model["id"])
                    row.append(entity.text)
                    row.append(entity.label_)
                    row.append(str(ent_freq))
                    rows.append(row)
                    # print(entity.text + ' - ' + 
                    #       entity.label_ + ' - ' + 
                    #       str(spacy.explain(entity.label_)) + ' - ' + 
                    #       str(ent_freq))
    except:
        print()
        print("error")
        print()    

Document  0 : 
--------------
Document  1 : 
--------------
Document  2 : 
--------------
Document  3 : 
--------------
Document  4 : 
--------------
Document  5 : 
--------------
Document  6 : 
--------------
Document  7 : 
--------------
Document  8 : 
--------------

error

Document  9 : 
--------------
Document  10 : 
--------------
Document  11 : 
--------------
Document  12 : 
--------------
Document  13 : 
--------------
Document  14 : 
--------------
Document  15 : 
--------------
Document  16 : 
--------------
Document  17 : 
--------------
Document  18 : 
--------------
Document  19 : 
--------------
Document  20 : 
--------------
Document  21 : 
--------------

error

Document  22 : 
--------------
Document  23 : 
--------------
Document  24 : 
--------------
Document  25 : 
--------------
Document  26 : 
--------------
Document  27 : 
--------------
Document  28 : 
--------------
Document  29 : 
--------------
Document  30 : 
--------------
Document  31 : 
--------------
Do

In [7]:
#df1 = df.sort_values('score',ascending = False).groupby('pidx').head(2)
import pickle

with open('docdict.json', 'wb') as fp:
    pickle.dump(docdict, fp)
    
with open('nlpmodels.json', 'wb') as fp:
    pickle.dump(models, fp)

# with open('docdict.json', 'rb') as fp:
#     data = pickle.load(fp)
    
# print(data)
# print(type(data))

In [11]:
import pandas as pd
row_dict = {}
for i in range(0, len(rows)):
    row_dict[i] = rows[i]

df = pd.DataFrame.from_dict(row_dict, orient='index', columns=['docid', 'entity', 'entity_type', 'entity_freq'])

In [12]:
df.to_csv("oecd-ner.csv", index=False)

In [24]:
df = pd.read_csv("oecd-ner.csv")
df2 = df.sort_values('entity_freq', ascending = False).groupby('docid').head(5)
df2 = df2.drop_duplicates(ignore_index=True)
df2.head(100)

,docid,entity,entity_type,entity_freq
0,26,oecd,ORG,858
1,26,oecd,GPE,858
2,47,oecd,ORG,568
3,47,oecd,GPE,568
4,52,oecd,ORG,519
...,...,...,...,...
75,29,m3,ORG,15
76,51,armenia,GPE,14
77,35,m3,ORG,12
78,39,switzerland,GPE,11


In [26]:
is_person =  df['entity_type']=="PERSON"
personsDf = df[is_person]
personsDf2 = personsDf.sort_values('entity_freq', ascending = False).groupby('docid').head(5)
personsDf2 = personsDf2.drop_duplicates(ignore_index=True)
personsDf2.head(50)

,docid,entity,entity_type,entity_freq
0,36,al.,PERSON,94
1,11,onas,PERSON,79
2,11,tunisia,PERSON,52
3,30,j.,PERSON,50
4,49,j.,PERSON,49
5,52,j.,PERSON,39
6,14,ebrd,PERSON,34
7,22,pes,PERSON,28
8,5,al.,PERSON,26
9,5,c.,PERSON,23
